In [1]:
from baselines.common import set_global_seeds, tf_util as U
from baselines import bench
import gym, logging
import roboschool
from baselines import logger
from baselines.ppo1 import mlp_policy, pposgd_simple
from base_line_model.mlp import MlpPolicy_new
from base_line_model.TRPO_agent import TRPO_agent_new

In [2]:
seed = 1
logger.configure()
U.make_session(num_cpu=16).__enter__()
set_global_seeds(seed)

Logging to /tmp/openai-2018-04-27-18-26-33-766730


# set up agents

In [3]:
env2 = gym.make("RoboschoolInvertedPendulum-v1")
env2 = bench.Monitor(env2, logger.get_dir(),allow_early_resets=True)
env2.seed(seed)
gym.logger.setLevel(logging.WARN)
class pargm(object):
    def __init__(self):
        self.timesteps_per_batch = 25000 # what to train on
        self.max_kl = 0.01
        self.cg_iters = 10
        self.gamma = 0.995
        self.lam =  0.97# advantage estimation
        self.entcoeff=0.0
        self.cg_damping=0.1
        self.vf_stepsize=1e-3
        self.vf_iters =5
        self.max_timesteps = 1e8
        self.max_episodes=0
        self.max_iters=0  # time constraint
        self.max_epi_avg = 1000
        self.callback=None


def policy_fn(name, ob_space, ac_space):
        return MlpPolicy_new(name=name, ob_space=ob_space, ac_space=ac_space,
            hid_size=128, num_hid_layers=2)
    
parg = pargm()
agent1 = TRPO_agent_new('pi1', env2, policy_fn, parg)

[2018-04-27 18:26:33,794] Making new env: RoboschoolInvertedPendulum-v1


Init param sum 1.89656


# Training

In [4]:
agent1.learn()

********** Iteration 0 ************
sampling
done in 26.756 seconds
computegrad
done in 0.150 seconds
cg
      iter residual norm  soln norm
         0      0.325          0
         1     0.0104     0.0433
         2    0.00165     0.0498
         3   0.000268      0.146
         4   5.16e-05      0.155
         5   6.26e-05       0.16
         6   1.81e-05      0.164
         7   3.04e-06      0.164
         8    1.2e-07      0.166
         9   9.07e-07      0.166
        10   1.45e-08      0.166
done in 0.365 seconds
Expected: 0.025 Actual: 0.025
Stepsize OK!
vf
done in 5.677 seconds
--------------------------------
| EpLenMean       | 21.6       |
| EpRewMean       | 21.6       |
| EpThisIter      | 1044       |
| EpisodesSoFar   | 1044       |
| TimeElapsed     | 33.2       |
| TimestepsSoFar  | 24991      |
| entloss         | 0.0        |
| entropy         | 1.41895    |
| ev_tdlam_before | -0.00662   |
| meankl          | 0.00913446 |
| optimgain       | 0.0251961  |
| surrgain

Expected: 0.005 Actual: 0.005
Stepsize OK!
vf
done in 5.087 seconds
--------------------------------
| EpLenMean       | 812        |
| EpRewMean       | 812        |
| EpThisIter      | 26         |
| EpisodesSoFar   | 2863       |
| TimeElapsed     | 233        |
| TimestepsSoFar  | 199669     |
| entloss         | 0.0        |
| entropy         | 1.41895    |
| ev_tdlam_before | 0.35       |
| meankl          | 0.00919307 |
| optimgain       | 0.00462797 |
| surrgain        | 0.00462797 |
--------------------------------
********** Iteration 8 ************
sampling
done in 22.747 seconds
computegrad
done in 0.075 seconds
cg
      iter residual norm  soln norm
         0    0.00236          0
         1   0.000428     0.0114
         2   0.000638     0.0203
         3   5.09e-06     0.0255
         4   1.06e-05     0.0272
         5   2.18e-06     0.0313
         6   5.23e-08      0.032
         7   3.02e-07     0.0322
         8   3.82e-09     0.0322
         9   1.13e-09     0.0322

# Evaluation

In [5]:
env = gym.make("RoboschoolInvertedPendulum-v1")
for i in range(10):
    score = 0
    obs = env.reset()
    done = False
    itr = 0
    do = False
    while done == False:   
        a = agent1.action_ev(obs)
        obs, r, done, _ = env.step(a)
        if done:
            do = True

        score += r
    print(score)

1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0


agent1.save_data('hopper_mem2')

In [6]:
#agent1.restore('hopper_mem2')